## Data

I think I need to figure out what I want to do with this first...

Probably start off with a model that predicts an ingredient list.

So I need to clean the ingredient data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /home/ec2-user/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [3]:
from src.data.pickling import load_pickle
import copy

In [4]:
ls data/raw/

shoestring_recipes.pickle  test.pickle


In [5]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [6]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [7]:
list(shoestring_recipes.keys())[:5]

['https://glutenfreeonashoestring.com/d-y-friday-homemade-hot-chocolate-mix/',
 'https://glutenfreeonashoestring.com/gluten-free-carrot-cake-cupcakes/',
 'https://glutenfreeonashoestring.com/gluten-free-strawberry-shortcake/',
 'https://glutenfreeonashoestring.com/',
 'https://glutenfreeonashoestring.com/vegan-gluten-free-muffins/']

In [10]:
urls = list(shoestring_recipes.keys())

In [75]:
recipe_instructions = {}

for recipe in shoestring_recipes:
    if shoestring_recipes[recipe] is not None:
        for key in shoestring_recipes[recipe].keys():
            instructions = ""
            for nbr in range(0,len(shoestring_recipes[recipe][key]["recipeInstructions"])-1):
                try:
                    if "text" in list(shoestring_recipes[recipe][key]["recipeInstructions"][nbr].keys()):
                        instructions += shoestring_recipes[recipe][key]["recipeInstructions"][nbr]['text'] + "\n"
                    elif "itemListElement" in list(shoestring_recipes[recipe][key]["recipeInstructions"][nbr].keys()):
                        for item in shoestring_recipes[recipe][key]["recipeInstructions"][nbr]["itemListElement"]:
                            instructions += item["text"] + "\n"
                except:
                    pass
            recipe_instructions[key] = instructions

In [ ]:
recipe_instructions

In [31]:
for recipe in shoestring_recipes:
    for instruction in shoestring_recipes[
        urls[0]
    ]['Homemade Gluten Free Hot Chocolate Mix [White Chocolate &amp; Dairy Free Options]']["recipeInstructions"]:
        if "text" in list(instruction.keys()):
            print(instruction['text'])
        elif "itemListElement" in list(instruction.keys()):
            for item in instruction["itemListElement"]:
                print(item["text"])

To make either mix, combine all the ingredients for that mix, except for the miniature marshmallows, in a large bowl and whisk to combine. Transfer each separately to a 16-ounce container with a lid, and cover until ready to use.
Store sealed at cool room temperature (a dark pantry works well) for up to 2 weeks until ready to use. For longer storage, store in the freezer.
Place 2 to 3 tablespoons of either mix in a mug. Heat the milk to about 180°F in a small saucepan or the microwave.
Stir the hot milk into the mug with the dry mix, whisking constantly until completely combined. Sprinkle with the optional miniature marshmallows and serve immediately.
This method produces a thinner hot chocolate, but in moments.
Place the 3 tablespoons of either mix and 1 cup milk in a small, heavy-bottom saucepan. Whisk to combine well.
Place over medium-high heat and bring to a boil, stirring constantly. Reduce the heat to medium and continue to simmer gently, stirring occasionally, until the mixture

In [521]:
recipe_dict = {}

for recipe in shoestring_recipes:
    try:
        for key in shoestring_recipes[recipe].keys():
            recipe_dict[key] = shoestring_recipes[recipe][key]["recipeIngredient"]
    except:
        pass

In [503]:
from src.features.clean_data import recipe_ingredient_dict

In [504]:
recipe_dict = recipe_ingredient_dict(shoestring_recipes)

In [ ]:
recipe_dict

In [ ]:
print(len(recipe_dict.keys()))
print(recipe_dict)

In [427]:
shoestring_recipes[
    'https://glutenfreeonashoestring.com/gluten-free-carrot-cake-cupcakes/'
]['Fluffy, Tender, and Easy Gluten Free Carrot Cake (or Cupcakes!)'
 ]["recipeIngredient"][:3]

['2 1/4 cups all purpose gluten free flour blend',
 '1 teaspoon xanthan gum (omit if your blend already contains it)',
 '1 teaspoon baking soda']

In [431]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# add numbers together
from fractions import Fraction

In [260]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [268]:
# for single recipe and single 
recipe_dict_test = copy.deepcopy(recipe_dict)

In [428]:
ingredients_pos = []

# POS tagging for ingredients list
for ingredient in list(recipe_dict_test.values())[1]:
    ingredients_pos.append(pos_tagging(ingredient))

In [430]:
ingredients_pos

[[('2', 'CD'),
  ('1/4', 'CD'),
  ('cups', 'NNS'),
  ('all', 'DT'),
  ('purpose', 'VBP'),
  ('gluten', 'NNS'),
  ('free', 'JJ'),
  ('flour', 'NNS'),
  ('blend', 'VBP')],
 [('1', 'CD'), ('teaspoon', 'NN'), ('xanthan', 'CD'), ('gum', 'NN')],
 [('1', 'CD'), ('teaspoon', 'NN'), ('baking', 'NN'), ('soda', 'NN')],
 [('1', 'CD'),
  ('1/2', 'CD'),
  ('teaspoons', 'NNS'),
  ('baking', 'VBG'),
  ('powder', 'NN')],
 [('1/2', 'CD'), ('teaspoon', 'NN'), ('kosher', 'NN'), ('salt', 'NN')],
 [('2', 'CD'), ('teaspoons', 'NNS'), ('ground', 'NN'), ('cinnamon', 'NN')],
 [('3/4', 'CD'), ('cup', 'NN'), ('granulated', 'VBD'), ('sugar', 'NN')],
 [('1/2', 'CD'),
  ('cup', 'NN'),
  ('packed', 'VBD'),
  ('light', 'JJ'),
  ('brown', 'NN'),
  ('sugar', 'NN')],
 [('1/2', 'CD'), ('cup', 'JJ'), ('neutral', 'JJ'), ('oil', 'NN')],
 [('4', 'CD'), ('eggs', 'NNS')],
 [('1', 'CD'),
  ('teaspoon', 'NN'),
  ('apple', 'NN'),
  ('cider', 'NN'),
  ('vinegar', 'NN')],
 [('1/2', 'CD'), ('cup', 'NN'), ('milk', 'NN')],
 [('1/2', 'C

In [353]:
a_dictionary = {"old_key": "value"}

a_dictionary["new_key"] = a_dictionary["old_key"]

print(a_dictionary)

{'old_key': 'value', 'new_key': 'value'}


In [514]:
recipe_dict_test = copy.deepcopy(recipe_dict)
len(recipe_dict_test)

142

In [523]:
clean_recipe_dict = {}

# clean recipe title of everything after any non-letter
for recipe in list(recipe_dict_test.keys()):
    if re.sub(r'[^a-zA-Z ]+', '', recipe) is not "":
        recipe_new = re.sub(r'[^a-zA-Z ]+', '', recipe)
    else:
        recipe_new = recipe
    clean_recipe_dict[recipe_new] = recipe_dict_test[recipe]

print(len(clean_recipe_dict))

142


In [524]:
clean_recipe_dict.keys()

dict_keys(['Homemade Gluten Free Hot Chocolate Mix White Chocolate amp Dairy Free Options', 'Fluffy Tender and Easy Gluten Free Carrot Cake or Cupcakes', 'Gluten Free Strawberry Shortcake', 'Vegan Gluten Free Muffins  Basic Recipe  MixIn Options', 'Gluten Free Brown Bread', 'Homemade Gluten Free Samoas Girl Scout Cookies', 'Master Gluten Free Scones Recipe', 'Super Fluffy Gluten Free Waffles', 'Classic Gluten Free Falafel Recipe  Healthy  Vegan', 'Gluten Free Cinnamon Rolls', 'Easy Delicious Gluten Free Stir Fry  Homemade Sauce  GF Rice Noodles', 'Homemade Gluten Free Spice Cake  Easy From Scratch Recipe', 'Thin amp Crispy Gluten Free Coconut Cookies Dairy Free Too', 'Gluten Free Hawaiian Rolls  with bread shaping videos', 'Flourless Chocolate Cookies  Crispy Chewy Naturally Gluten Free', 'Easy Gluten Free White Sandwich Bread Recipe  Tender and Springy', 'Homemade Protein Bars', 'Gluten Free Apple Bread  Moist Spicy and Tender with Apple Butter', 'Basic Gluten Free Pizza Dough', 'Clas

In [516]:
new_recipe_dict[
    'Fluffy Tender and Easy Gluten Free Carrot Cake or Cupcakes'
][-1]

'4 cups confectioners’ sugar (plus more as necessary)'

In [517]:
# cleans ingredients
# TODO: remove apostrophes - isn't working so maybe not an apostrophe??
# might have to clean it by hand
for recipe in clean_recipe_dict:
    clean_recipe_dict[recipe] = [
        ingredient.split("(")[0].rstrip() for ingredient in clean_recipe_dict[recipe]
    ]
    clean_recipe_dict[recipe] = [
        ingredient.replace("'", "") for ingredient in clean_recipe_dict[recipe]
    ]

In [518]:
clean_recipe_dict[
    'Fluffy Tender and Easy Gluten Free Carrot Cake or Cupcakes'
][-1]

'4 cups confectioners’ sugar'

In [ ]:
# POS tagging for ingredients list
recipe_dict_test = copy.deepcopy(clean_recipe_dict)

In [450]:
ingredients_pos = {}

for recipe in recipe_dict_test:
    ingredients_list_pos = []

    for ingredient in recipe_dict_test[recipe]:
        ingredients_list_pos.append(pos_tagging(ingredient))

    ingredients_pos[recipe] = ingredients_list_pos

In [ ]:
print(len(ingredients_pos.keys()))
print(ingredients_pos)

In [456]:
ingredients_pos = {}

for recipe in clean_recipe_dict:
    ingredients_list_pos = []

    for ingredient in clean_recipe_dict[recipe]:
        ingredients_list_pos.append(pos_tagging(ingredient))

    # For each ingredient in an ingredients list
    for ingredient in range(0,len(ingredients_list_pos)):
        remove_index = []
        final_nbr = 0
        sum_nbr = 0
        nbr_cd = 0
        # For each POS of an ingredient
        for pos in ingredients_list_pos[ingredient]:
            # If it's a quantity
            if pos[1] == "CD":
                # Get index of POS
                pos_index = ingredients_list_pos[ingredient].index(pos)
                # Turn quantity to a real number
                try:
                    final_nbr += Fraction(pos[0])
                    round_nbr = round(
                        final_nbr.numerator / final_nbr.denominator, 2
                    )
                    new_pos = (round_nbr, "CD")
                    # Set old POS as new
                    ingredients_list_pos[ingredient][pos_index] = new_pos
                    # Track number of POS quantities related to this ingredient
                    nbr_cd += 1
                    # If multiple quantity POS's related to ingredient
                    if nbr_cd > 1:
                        # Remove previous quantity
                        ingredients_list_pos[ingredient].pop(pos_index-1)
                except:
                    pass

    ingredients_pos[recipe] = ingredients_list_pos

In [497]:
my_recipe = ingredients_pos["Fluffy Tender and Easy Gluten Free Carrot Cake or Cupcakes"]
recipe_pos = {}

for recipe in ingredients_pos:
    ingredients = []

    for ingredient in ingredients_pos[recipe]:
        ingredient_full = []
        for item in ingredient:
            ingredient_full.append(str(item[0]))

        ingredients.append(" ".join(ingredient_full).rstrip())

    recipe_pos[recipe] = ingredients

print(recipe_pos["Fluffy Tender and Easy Gluten Free Carrot Cake or Cupcakes"])

['2.25 cups all purpose gluten free flour blend', '1.0 teaspoon xanthan gum', '1.0 teaspoon baking soda', '1.5 teaspoons baking powder', '0.5 teaspoon kosher salt', '2.0 teaspoons ground cinnamon', '0.75 cup granulated sugar', '0.5 cup packed light brown sugar', '0.5 cup neutral oil', '4.0 eggs', '1.0 teaspoon apple cider vinegar', '0.5 cup milk', '0.5 cup chopped pecans', '0.5 cup white chocolate chips', '3.0 cups peeled and grated carrots', '8.0 ounces cream cheese', '8.0 tablespoons unsalted butter', '0.25 teaspoon kosher salt', '4.0 cups confectioners ’ sugar']


In [532]:
from src.features.clean_data import (clean_recipe_title, recipe_ingredient_dict,
                                     clean_recipe_ingredients, format_ingredients, pos_tuples_to_list)

In [539]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [529]:
recipe_dict = recipe_ingredient_dict(shoestring_recipes)
print(recipe_dict['Homemade Gluten Free Hot Chocolate Mix [White Chocolate &amp; Dairy Free Options]'])

['3/4 cup unsweetened cocoa powder ((preferably Dutch-processed))', '1/2 cup powdered milk', '2 teaspoons gluten free sweet rice flour', '1/2 teaspoon kosher salt', '3/4 cup granulated sugar', '2 ounces bittersweet chocolate, at cool room temperature (finely grated or pulsed in blender or food processor (optional))', '1/3 cup powdered milk', '1 1/2 teaspoons gluten free sweet rice flour', '1/2 teaspoon kosher salt', '1/4 cup granulated sugar', '4 ounces white chocolate, at cool room temperature (finely grated or pulsed in blender or food processor)', '3 tablespoons dry mix', '1 cup milk', 'Miniature marshmallows (for serving (optional))']


In [530]:
recipe_dict_clean_titles = clean_recipe_title(recipe_dict)
print(recipe_dict_clean_titles.keys())

In [533]:
recipe_dict_clean_ingredients = clean_recipe_ingredients(recipe_dict_clean_titles)
print(recipe_dict_clean_ingredients["Homemade Gluten Free Hot Chocolate Mix White Chocolate amp Dairy Free Options"])

In [ ]:
recipe_dict_format_ingredients = format_ingredients(recipe_dict_clean_ingredients)
print(recipe_dict_format_ingredients["Homemade Gluten Free Hot Chocolate Mix White Chocolate amp Dairy Free Options"])

In [536]:
recipe_dict_ingredients_list = pos_tuples_to_list(recipe_dict_format_ingredients)
print(recipe_dict_ingredients_list["Homemade Gluten Free Hot Chocolate Mix White Chocolate amp Dairy Free Options"])

['0.75 cup unsweetened cocoa powder', '0.5 cup powdered milk', '2.0 teaspoons gluten free sweet rice flour', '0.5 teaspoon kosher salt', '0.75 cup granulated sugar', '2.0 ounces bittersweet chocolate , at cool room temperature', '0.33 cup powdered milk', '1.5 teaspoons gluten free sweet rice flour', '0.5 teaspoon kosher salt', '0.25 cup granulated sugar', '4.0 ounces white chocolate , at cool room temperature', '3.0 tablespoons dry mix', '1.0 cup milk', 'Miniature marshmallows']


In [537]:
len(recipe_dict_ingredients_list)

142

In [541]:
from src.features.clean_shoestring_data import clean_shoestring_recipes

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [542]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

# Next steps

- See what form data needs to be in for model and do that
- Remove untagged words

POS tag list:

POS_tag | definiton
- | -
CC | coordinating conjunction
CD | cardinal digit
DT | determiner
EX | existential there (like: "there is" ... think of it like "there exists")
FW | foreign word
IN | preposition/subordinating conjunction
JJ | adjective 'big'
JJR | adjective, comparative 'bigger'
JJS | adjective, superlative 'biggest'
LS | list marker 1)
MD | modal could, will
NN | noun, singular 'desk'
NNS | noun plural 'desks'
NNP | proper noun, singular 'Harrison'
NNPS | proper noun, plural 'Americans'
PDT | predeterminer 'all the kids'
POS | possessive ending parent's
PRP | personal pronoun I, he, she
PRP$ | possessive pronoun my, his, hers
RB | adverb very, silently,
RBR | adverb, comparative better
RBS | adverb, superlative best
RP | particle give up
TO | to go 'to' the store.
UH | interjection errrrrrrrm
VB | verb, base form take
VBD | verb, past tense took
VBG | verb, gerund/present participle taking
VBN | verb, past participle taken
VBP | verb, sing. present, non-3d take
VBZ | verb, 3rd person sing. present takes
WDT | wh-determiner which
WP | wh-pronoun who, what
WP$ | possessive wh-pronoun whose
WRB | wh-abverb where, when